In [1]:
def clamp(n, minimum, maximum):
    return max(min(maximum, n), minimum)

In [2]:
#This cell organizes the data for renewable installations
import csv, re, math
from collections import defaultdict

renewable = {}
with open('data/Renewable Energy Installations.csv', encoding='utf-8') as f:
    csv_reader = csv.DictReader(f)

    for line in csv_reader:
        name = line['LINEAR_NAME_FULL']
        geo = line['geometry']
        
        renewable.setdefault(name, {
            'Latitude': 0,
            'Longitude': 0
        })
        
        #Gathers coordinates of renewables
        x = re.search(r"\(([A-Za-z0-9.\-,\s]+)\)", geo)
        coordinates = x.group(1).split(", ")
        
        renewable[name]['Latitude'] = coordinates[1]
        renewable[name]['Longitude'] = coordinates[0]

for i, value in renewable.items():
    #print(i)
    a = renewable[i]['Latitude']
    b = renewable[i]['Longitude']
    #print(f'\t{a}, {b}')

In [10]:
#This cell organizes the data for CO2 emissions
import csv, pgeocode
from collections import defaultdict

building = {}

with open('data/Annual Energy.csv', encoding='utf-8') as f:
    csv_reader = csv.DictReader(f)
    
    for line in csv_reader:
        name = line['\ufeffOperation Name']
        if not name == "":
            if (not line['Total (calculated in webform)'].replace(",","") == ""):
                emissions = int(line['Total (calculated in webform)'].replace(",",""))
            else:
                emissions = 0
            postal = line['Postal Code']
        
            nomi = pgeocode.Nominatim('ca')
        
            building.setdefault(name, {
                'Emissions': 0,
                'Latitude': 0,
                'Longitude': 0
            })
        
            nomi = nomi.query_postal_code(postal)
            #Sets Coordinates
            building[name]['Longitude'] = (nomi.longitude)
            building[name]['Latitude'] = (nomi.latitude)
            building[name]['Emissions'] = emissions
            
#Sorts dictionary by highest to lowest emissions so on the map the big circle appears below
building = dict(sorted(building.items(), key = lambda kv: kv[1]['Emissions'], reverse=True))

for place, value in building.items():
    a = building[place]['Longitude']
    b = building[place]['Latitude']
    c = building[place]['Emissions']
    
    #print(place)
    #print(f'\t Coordinates:{a}, {b} - Emissions:{c}')

In [19]:
#This cell organizes data for green roofs
import csv, pgeocode
from collections import defaultdict

roof = {}

with open ('data/Green Roofs.csv', encoding='utf-8') as f:
    csv_reader = csv.DictReader(f)
    
    for line in csv_reader:
        complete = line["COMPLETED_DATE"]
        postal = line["POSTAL"]
        if not complete == "" and not postal == "   ":
            street = line["STREET_NAME"]
            
            roof.setdefault(street, {
                'Area': 0,
                'Latitude': 0,
                'Longitude': 0
            })
            
            nomi = pgeocode.Nominatim('ca')
            nomi = nomi.query_postal_code(postal)
            roof[street]['Longitude'] = (nomi.longitude)
            roof[street]['Latitude'] = (nomi.latitude)
            #building[street]['Area'] = 5


In [ ]:
#This cell maps renewables vs GHG emissions
from ipyleaflet import Map, Marker, basemaps, Popup, Circle
from ipywidgets import HTML

highEmission = 3000000

#Creates map
m = Map(center = (43.6487, -79.38544), zoom = 10, min_zoom = 1, max_zoom = 20, 
    basemap=basemaps.Stamen.Terrain)
#m = Map(center = (43.6656, -79.383), zoom = 10, min_zoom = 1, max_zoom = 20)

#Adds markers
#https://ipyleaflet.readthedocs.io/en/latest/api_reference/popup.html
for i, value in renewable.items():
    #Stores coordinates in integers
    a = renewable[i]['Latitude']
    b = renewable[i]['Longitude']
    
    #Adds all the markers
    center = (a, b)
    marker = Marker(location=center, draggable=False)
    m.add_layer(marker);
    
    #Creates message
    message = HTML()
    message.value = i
    
    #Creates popup
    popup = Popup(
        location=center,
        close_button=False,
        auto_close=False,
        close_on_escape_key=False
    )
    #Adds popup to layer and associates it with the message and marker
    marker.popup = message
    
# Adds circle markers
for place, value in building.items():
    circle = Circle()
    circle.location = (building[place]['Latitude'], building[place]['Longitude'])
    ghg = building[place]['Emissions']
    circle.radius = int(clamp(math.sqrt(ghg), 50, 1750))
    #Calculates the color based on greenhouse gas emissions, higher means more red
    green = str(hex(int(clamp(highEmission - ghg, 0, highEmission/2) * 2 * 255/highEmission)))
    if int(green, 16) < 16:
        green = "0" + green
    green = green.replace("0x", "")
    
    red = str(hex(int(clamp(ghg, 0, highEmission/2) * 2 * 255/highEmission)))
    if int(red,16) < 16:
        red = "0" + red
    red = red.replace("0x", "")
    
    col = "#" + red + green + "00"
    circle.color = col
    circle.fill_color = col
    
    #Creates a popup for each circle
    popup2 = Popup(
        location=(building[place]['Latitude'], building[place]['Longitude']),
        close_button=False,
        auto_close=False,
        close_on_escape_key=False
    )
    
    #Creates message, which shows division and percentage
    message2 = HTML()
    name = place
    message2.value = (f'\t{name}, {ghg} kg')
    
    m.add_layer(circle)
    circle.popup = message2

#Draws Map
m

<h1>Renewables Analysis<h1>
    

In [34]:
#This cell stores information on the age and income of people in the survey and whether they deny climate change
import csv, pgeocode
from collections import defaultdict

person = {}

with open ('data/Climate Perceptions.csv', encoding='utf-8') as f:
    csv_reader = csv.DictReader(f)
    
    for line in csv_reader:
        record = line["\ufeffrecord"]
        age = line["HIDAGE1"]
        postal = line["QS1"]
        concern = line["Q1r2"]
        knowledge = line["Q2"]
    
        if concern == "Extremely concerned":
            concern = 5
        elif concern == "Very concerned":
            concern = 4
        elif concern == "Don't know":
            concern = 3
        elif concern == "Not very concerned":
            concern = 2
        else:
            concern = 1
        
        if knowledge == "Extremely informed":
            knowledge = 4
        elif knowledge == "Very informed":
            knowledge = 3
        elif knowledge == "Not very informed":
            knowledge = 2
        else:
            knowledge = 1
        
        person.setdefault(record, {
            'Age': 0,
            'Concern': 0,
            'Knowledge': 0,
            'Latitude': 0,
            'Longitude': 0
        })
    